# 환경 설정

In [1]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.5 MB/s eta 0:00:00


# 데이터 압축 해제

In [2]:
# zip 파일 읽어서 압축 해제

import os
import zipfile

# 경로 설정
base_dir = "/content/YOLO_safety"   # colab 로컬 디렉토리
os.makedirs(base_dir, exist_ok=True)

zip_path = "/content/drive/MyDrive/AI/"

# 압축 해제 함수
def unzip_files(zip_path, target_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(target_dir)
    print(f"압축 해제 완료: {zip_path} -> {target_dir}")

# 압축 해제
file_name = "1_bbox_completed.zip"
unzip_files(zip_path + file_name, base_dir)

# 압축 해제
file_name = "2_bbox_completed.zip"
unzip_files(zip_path + file_name, base_dir)

압축 해제 완료: /content/drive/MyDrive/AI/1_bbox_completed.zip -> /content/YOLO_safety
압축 해제 완료: /content/drive/MyDrive/AI/2_bbox_completed.zip -> /content/YOLO_safety


# JSON 라벨을 YOLO 포맷으로 변환

In [3]:
import json
from pathlib import Path

def convert_box_json_to_yolo(json_dir, output_dir, class_map):
    """
    box 또는 polygon 기반 JSON 라벨을 YOLO 포맷으로 변환하는 함수.
    polygon은 좌표의 min/max로 bounding box를 추정.
    """
    json_dir = Path(json_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    json_files = list(json_dir.glob("*.json"))
    print(f"총 {len(json_files)}개의 JSON 파일을 처리합니다.")

    converted_count = 0  # 실제 변환된 JSON 수

    for json_file in json_files:
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)

        filename = data["image"]["filename"]
        img_width, img_height = data["image"]["resolution"]
        label_lines = []

        for ann in data.get("annotations", []):
            class_id_raw = ann.get("class")
            if class_id_raw not in class_map:
                continue

            class_id = class_map[class_id_raw]

            # 1. bbox가 있으면 우선 사용
            box = ann.get("box")
            if box and len(box) == 4:
                xmin, ymin, xmax, ymax = box
            # 2. polygon만 있는 경우
            elif "polygon" in ann and len(ann["polygon"]) >= 3:
                xs = [pt[0] for pt in ann["polygon"]]
                ys = [pt[1] for pt in ann["polygon"]]
                xmin, xmax = min(xs), max(xs)
                ymin, ymax = min(ys), max(ys)
            else:
                continue    # 유효한 box도 polygon도 없는 경우

            cx = ((xmin + xmax) / 2) / img_width
            cy = ((ymin + ymax) / 2) / img_height
            w = (xmax - xmin) / img_width
            h = (ymax - ymin) / img_height

            label_lines.append(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")

        if label_lines:
            label_file = output_dir / (Path(filename).stem + ".txt")
            with open(label_file, "w") as out_f:
                out_f.write("\n".join(label_lines))
            converted_count += 1  # ✅ 변환된 경우만 카운트 증가

    print(f"✅ YOLO 라벨로 변환 완료: {converted_count}개 파일 생성됨")

In [4]:
# 클래스 매핑: JSON 클래스 번호(str) → YOLO 클래스 인덱스(int)
class_mapping_all = {
    "01": 0,  # 안전벨트 착용
    "03": 1,  # 안전고리 결착
    "07": 2,  # 안전모 착용
}

# ✅ 클래스별로 반복 처리
for cls_str, yolo_idx in class_mapping_all.items():
    # 학습 데이터
    convert_box_json_to_yolo(
        json_dir=f"/content/YOLO_safety/train/labels_json/{cls_str}",
        output_dir=f"/content/YOLO_safety/train/labels",
        class_map={cls_str: yolo_idx}  # 해당 클래스만 변환
    )
    print(f"클래스: {cls_str} 학습 데이터 라벨 변환 완료")

    # 검증 데이터
    convert_box_json_to_yolo(
        json_dir=f"/content/YOLO_safety/val/labels_json/{cls_str}",
        output_dir=f"/content/YOLO_safety/val/labels",
        class_map={cls_str: yolo_idx}  # 해당 클래스만 변환
    )
    print(f"클래스: {cls_str} 검증 데이터 라벨 변환 완료\n")

총 1515개의 JSON 파일을 처리합니다.
✅ YOLO 라벨로 변환 완료: 1515개 파일 생성됨
클래스: 01 학습 데이터 라벨 변환 완료
총 256개의 JSON 파일을 처리합니다.
✅ YOLO 라벨로 변환 완료: 256개 파일 생성됨
클래스: 01 검증 데이터 라벨 변환 완료

총 691개의 JSON 파일을 처리합니다.
✅ YOLO 라벨로 변환 완료: 691개 파일 생성됨
클래스: 03 학습 데이터 라벨 변환 완료
총 177개의 JSON 파일을 처리합니다.
✅ YOLO 라벨로 변환 완료: 177개 파일 생성됨
클래스: 03 검증 데이터 라벨 변환 완료

총 1637개의 JSON 파일을 처리합니다.
✅ YOLO 라벨로 변환 완료: 1637개 파일 생성됨
클래스: 07 학습 데이터 라벨 변환 완료
총 257개의 JSON 파일을 처리합니다.
✅ YOLO 라벨로 변환 완료: 257개 파일 생성됨
클래스: 07 검증 데이터 라벨 변환 완료



In [5]:
from pathlib import Path

train_label_dir = Path("/content/YOLO_safety/train/labels")
val_label_dir = Path("/content/YOLO_safety/val/labels")

train_count = len(list(train_label_dir.glob("*.txt")))
val_count = len(list(val_label_dir.glob("*.txt")))

print(f"✅ 총 라벨 파일 개수: {train_count + val_count}개")
print(f" - 학습: {train_count}개")
print(f" - 검증: {val_count}개")

✅ 총 라벨 파일 개수: 4517개
 - 학습: 3839개
 - 검증: 678개


In [6]:
import os
from collections import Counter

def count_yolo_classes(label_dir, class_names=None):
    """
    YOLO 라벨(.txt) 파일에서 클래스별 인스턴스 수를 카운트합니다.

    :param label_dir: 라벨 파일(.txt)들이 있는 디렉토리
    :param class_names: 클래스 이름 리스트 (있으면 이름과 함께 출력됨)
    """
    counter = Counter()

    for filename in os.listdir(label_dir):
        if filename.endswith(".txt"):
            with open(os.path.join(label_dir, filename), "r", encoding="utf-8") as f:
                for line in f:
                    class_id = line.strip().split()[0]  # 첫 번째 값이 class_id
                    counter[int(class_id)] += 1

    print("📊 클래스별 인스턴스 수:")
    for class_id, count in sorted(counter.items()):
        name = class_names[class_id] if class_names else f"class_{class_id}"
        print(f"  {name} ({class_id}): {count}개")

    return counter

label_dir = "/content/YOLO_safety/train/labels"  # 또는 val
class_names = ["safety_harness", "safety_lanyard", "safety_helmet"]
count_yolo_classes(label_dir, class_names)

📊 클래스별 인스턴스 수:
  safety_harness (0): 3353개
  safety_lanyard (1): 711개
  safety_helmet (2): 2790개


Counter({2: 2790, 0: 3353, 1: 711})

# 이미지 경로 평탄화

In [7]:
import shutil
from pathlib import Path

def flatten_image_dir_skip_duplicates(img_base_dir: Path):
    flat_img_dir = img_base_dir

    for subdir in img_base_dir.iterdir():
        if subdir.is_dir():
            for img_file in subdir.glob("*.jpg"):
                target_path = flat_img_dir / img_file.name

                if target_path.exists():
                    print(f"⚠️ 중복으로 건너뜀: {img_file.name}")
                    continue

                shutil.move(str(img_file), str(target_path))

    print(f"✅ 중복 건너뛰며 평탄화 완료: {img_base_dir}")

# 적용
flatten_image_dir_skip_duplicates(Path("/content/YOLO_safety/train/images"))
flatten_image_dir_skip_duplicates(Path("/content/YOLO_safety/val/images"))

⚠️ 중복으로 건너뜀: S2-N1206M03139.jpg
⚠️ 중복으로 건너뜀: S2-N1206M06642.jpg
⚠️ 중복으로 건너뜀: S2-N1206M06925.jpg
⚠️ 중복으로 건너뜀: S2-N1206M03006.jpg
✅ 중복 건너뛰며 평탄화 완료: /content/YOLO_safety/train/images
⚠️ 중복으로 건너뜀: S2-N0109M37690.jpg
⚠️ 중복으로 건너뜀: S2-N0108M06974.jpg
⚠️ 중복으로 건너뜀: S2-N0109M18719.jpg
⚠️ 중복으로 건너뜀: S2-N0109M07195.jpg
⚠️ 중복으로 건너뜀: S2-N0109M38884.jpg
⚠️ 중복으로 건너뜀: S2-N0109M51805.jpg
⚠️ 중복으로 건너뜀: S2-N0109M36558.jpg
⚠️ 중복으로 건너뜀: S2-N0109M14633.jpg
⚠️ 중복으로 건너뜀: S2-N0108M20649.jpg
⚠️ 중복으로 건너뜀: S2-N0108M04256.jpg
⚠️ 중복으로 건너뜀: S2-N2103M34449.jpg
⚠️ 중복으로 건너뜀: S2-N0108M06791.jpg
✅ 중복 건너뛰며 평탄화 완료: /content/YOLO_safety/val/images


# 데이터 존재 확인

In [19]:
from pathlib import Path

# 이미지 디렉토리 존재 확인
print("✅ train images exists:", Path("/content/YOLO_safety/train/images").exists())
print("✅ val images exists:", Path("/content/YOLO_safety/val/images").exists())

# 라벨 디렉토리 + 파일 개수 확인
train_labels = list(Path("/content/YOLO_safety/train/labels").glob("*.txt"))
print(f"✅ train 라벨 수: {len(train_labels)}개")

val_labels = list(Path("/content/YOLO_safety/val/labels").glob("*.txt"))
print(f"✅ val 라벨 수: {len(val_labels)}개")

# 비어있는 파일 확인
empty = [f for f in val_labels if f.stat().st_size == 0]
print(f"⚠️ 비어있는 라벨 파일 수: {len(empty)}개")

✅ train images exists: True
✅ val images exists: True
✅ train 라벨 수: 3839개
✅ val 라벨 수: 678개
⚠️ 비어있는 라벨 파일 수: 0개


In [20]:
from pathlib import Path

def check_image_label_match(images_dir: Path, labels_dir: Path):
    # 이미지 파일 이름(확장자 제거)
    image_files = [f.stem for f in images_dir.glob("*.*")]
    label_files = [f.stem for f in labels_dir.glob("*.txt")]

    image_set = set(image_files)
    label_set = set(label_files)

    # 라벨 없는 이미지
    no_label = image_set - label_set
    # 이미지 없는 라벨
    no_image = label_set - image_set

    print(f"=== {images_dir.parent.name.upper()} 세트 ===")
    print(f"총 이미지: {len(image_files)}, 총 라벨: {len(label_files)}")
    print(f"⚠️ 라벨 없는 이미지: {len(no_label)}개")
    print(f"⚠️ 이미지 없는 라벨: {len(no_image)}개\n")

    if no_label:
        print("라벨 없는 이미지 예시:", list(no_label)[:5])
    if no_image:
        print("이미지 없는 라벨 예시:", list(no_image)[:5])


# 경로 설정
train_images_path = Path("/content/YOLO_safety/train/images")
train_labels_path = Path("/content/YOLO_safety/train/labels")
val_images_path = Path("/content/YOLO_safety/val/images")
val_labels_path = Path("/content/YOLO_safety/val/labels")

# train / val 각각 검사
check_image_label_match(train_images_path, train_labels_path)
check_image_label_match(val_images_path, val_labels_path)

=== TRAIN 세트 ===
총 이미지: 3839, 총 라벨: 3839
⚠️ 라벨 없는 이미지: 0개
⚠️ 이미지 없는 라벨: 0개

=== VAL 세트 ===
총 이미지: 678, 총 라벨: 678
⚠️ 라벨 없는 이미지: 0개
⚠️ 이미지 없는 라벨: 0개



In [21]:
# 잘못된 클래스 ID가 있는지 확인
bad_lines = 0
for f in val_labels:
    lines = f.read_text().splitlines()
    for line in lines:
        parts = line.strip().split()
        if not parts:
            continue
        try:
            class_id = int(parts[0])
            if class_id not in [0, 1, 2]:
                print(f"⚠️ 잘못된 클래스 ID: {class_id} in {f.name}")
                bad_lines += 1
        except:
            print(f"⚠️ 형식 오류 in {f.name}: {line}")
            bad_lines += 1

print(f"❗ 잘못된 라벨 줄 수: {bad_lines}")

❗ 잘못된 라벨 줄 수: 0


In [22]:
# 라벨 파일 몇 개 출력해보기 (예: 처음 3개)
for f in val_labels[:3]:
    print(f"\n📄 파일명: {f.name}")
    print(f.read_text())


📄 파일명: S2-N0108M17275.txt
1 0.671354 0.478704 0.151042 0.085185

📄 파일명: S2-N1106M20523.txt
1 0.586198 0.254167 0.120313 0.376852

📄 파일명: S2-N1106M19758.txt
1 0.542448 0.462037 0.064062 0.287037


In [23]:
val_img_dir = Path("/content/YOLO_safety/val/images")
val_images = [f.stem for f in val_img_dir.glob("*.jpg")]
val_labels = [f.stem for f in Path("/content/YOLO_safety/val/labels").glob("*.txt")]

no_label_images = set(val_images) - set(val_labels)

print(f"⚠️ 라벨이 없는 이미지 수: {len(no_label_images)}")
print(list(no_label_images)[:5])  # 일부 예시

⚠️ 라벨이 없는 이미지 수: 0
[]


In [24]:
import os

# 존재 확인
print("train 이미지 디렉토리 있음:", os.path.exists("/content/YOLO_safety/train/images"))
print("train 라벨 디렉토리 있음:", os.path.exists("/content/YOLO_safety/train/labels"))
print("val 이미지 디렉토리 있음:", os.path.exists("/content/YOLO_safety/val/images"))
print("val 라벨 디렉토리 있음:", os.path.exists("/content/YOLO_safety/val/labels"))

train 이미지 디렉토리 있음: True
train 라벨 디렉토리 있음: True
val 이미지 디렉토리 있음: True
val 라벨 디렉토리 있음: True


# 데이터 증강

In [ ]:
hyp_text = """
# /content/YOLO_safety/hyp_cctv.yaml

# 증강 - 좌우/상하 반전
fliplr: 0.3             # 좌우 반전 확률 (30%) ← CCTV에서 좌우 반전은 어느 정도 허용
flipud: 0.0             # 상하 반전 확률 (0%) ← CCTV는 고정 앵글이므로 비권장

# 증강 - Mosaic & Mixup
mosaic: 0.5             # mosaic 증강 확률 (기본 1.0, CCTV에서는 절반만 적용)
mixup: 0.0              # mixup 비활성화 ← CCTV에서는 부적절할 수 있음

# 증강 - 색상 계열 (HSV)
hsv_h: 0.015            # 색상 변화 범위 (Hue shift)
hsv_s: 0.7              # 채도 변화 범위 (Saturation shift)
hsv_v: 0.4              # 명도 변화 범위 (Value shift)

# 증강 - 기하학 왜곡
degrees: 0.0            # 회전 각도 (0도) ← CCTV 고정 카메라에선 회전 비권장
translate: 0.2          # 평행 이동 범위 (20%)
scale: 0.2              # 스케일 조정 범위 (20%)
shear: 0.0              # 기울이기 (Shear) 적용 안함
perspective: 0.0        # 원근 왜곡 비활성화 ← CCTV에서는 적용하지 않음
"""

with open("/content/YOLO_safety/hyp_cctv.yaml", "w") as f:
    f.write(hyp_text)

print("✅ hyp_cctv.yaml 저장 완료!")

✅ hyp_cctv.yaml 저장 완료!


# 학습

In [ ]:
yaml_text = """
path: /content/YOLO_safety
train: train/images
val: val/images

names:
  0: safety_harness
  1: safety_lanyard
  2: safety_helmet
"""

with open("/content/YOLO_safety/safety.yaml", "w") as f:
    f.write(yaml_text)

print("safety.yaml 저장 완료!")

safety.yaml 저장 완료!


## 증강 적용 X

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")  # 또는 yolov8s.pt -> yolov8n.pt -> yolov8m.pt

model.train(
    data="/content/YOLO_safety/safety.yaml",            # 데이터셋 구성정보가 담긴 yaml
    epochs=50,                                          # 전체 학습 반복 횟수
    imgsz=640,                                          # 학습에 사용할 입력 이미지 크기
    project="/content/drive/MyDrive/YOLO_safety/runs",  # 실험 결과 저장 경로
    name="safetyGear_detect_exp1"                       # 실험명
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 354MB/s]


Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/YOLO_safety/safety.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=safetyGear_detect_exp17, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True,

100%|██████████| 755k/755k [00:00<00:00, 90.5MB/s]

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        690       1257       0.54      0.665      0.519      0.341



50 epochs completed in 1.880 hours.
Optimizer stripped from /content/drive/MyDrive/YOLO_safety/runs/safetyGear_detect_exp17/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/YOLO_safety/runs/safetyGear_detect_exp17/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/YOLO_safety/runs/safetyGear_detect_exp17/weights/best.pt...
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:14<00:00,  1.56it/s]


                   all        690       1257      0.534      0.707      0.547      0.356
        safety_harness        247        518      0.473       0.79      0.465      0.265
        safety_lanyard        174        188      0.637      0.681      0.617      0.424
         safety_helmet        257        551      0.491      0.652      0.557      0.379
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to /content/drive/MyDrive/YOLO_safety/runs/safetyGear_detect_exp17


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f3e14fa4ed0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

## 증강 적용 O

In [ ]:
# import yaml

# # hyp.yaml을 딕셔너리로 로드
# with open("/content/YOLO_safety/hyp_cctv.yaml", "r") as f:
#     hyp_dict = yaml.safe_load(f)

In [ ]:
# from ultralytics import YOLO
# model = YOLO("yolov8n.pt")  # 또는 yolov8s.pt -> yolov8n.pt -> yolov8m.pt

# model.train(
#     data="/content/YOLO_safety/safety.yaml",            # 데이터셋 구성정보가 담긴 yaml
#     epochs=50,                                          # 전체 학습 반복 횟수
#     imgsz=640,                                          # 학습에 사용할 입력 이미지 크기
#     project="/content/drive/MyDrive/YOLO_safety/runs",  # 실험 결과 저장 경로
#     name="safetyGear_detect_exp1",                      # 실험명
#     **hyp_dict                                          # 내부 증강 및 하이퍼파라미터 설정 yaml
# )
